In [13]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import naive_bayes 
from sklearn import preprocessing

In [4]:
#breaing up x and y data sets
data = pd.read_csv("C:/Users/USERID/Documents/GitHub/Employment2030/Employment2030/tables/scores_answers.csv",index_col=['noc','workshop.number'])
data.sort_index(inplace=True)
data.loc[data.share == 'remain constant','share'] = 'constant'
data.dropna(inplace=True)
x = data.drop(['absolute','share','Unnamed: 0','noc_code'],axis=1) #making x data frame
x['work_num'] = x.index.get_level_values(1) #making workshop number a variable as well as an index
y_abs = data['absolute'].copy() #y - absolute change
y_share = data['share'].copy() #y - proportional change

In [5]:
#answer proportions to check learned probabillities againts
#abs_answer = data.pivot_table(index = data.index, columns = 'absolute', aggfunc = len)
abs_answer = pd.DataFrame(y_abs).pivot_table(index = ['noc','workshop.number'], columns = 'absolute', aggfunc = len).fillna(0)
abs_answer['sum'] = abs_answer.sum(axis = 1)
abs_answer['not_increase'] = abs_answer['fewer'] + abs_answer['same']
abs_answer.loc[:,abs_answer.columns!='sum'] = abs_answer.loc[:,abs_answer.columns!='sum'].divide(abs_answer['sum'],axis=0)

share_answer = pd.DataFrame(y_share).pivot_table(index = ['noc','workshop.number'], columns = 'share', aggfunc = len).fillna(0)
share_answer['sum'] = share_answer.sum(axis = 1)
share_answer['not_increase'] = share_answer['decrease'] + share_answer['constant']
share_answer.loc[:,share_answer.columns!='sum'] = share_answer.loc[:,share_answer.columns!='sum'].divide(share_answer['sum'],axis=0)

In [6]:
#making a categorical x
enc = preprocessing.OneHotEncoder(sparse = False)
x_onehot = np.round(x).astype(str)
enc.fit(x_onehot)
x_onehot = enc.transform(x_onehot)
#super inefficent but kind of convient, probably shouldn't do this
x_onehot = pd.DataFrame(x_onehot,columns=enc.get_feature_names(x.columns),index=x.index) 

#rounded x (for multimial distribution)
x_round = np.round(x)

#binary y's
y_abs_bin = y_abs.copy()
y_abs_bin.loc[y_abs_bin.isin(['fewer','same'])] = 'fewer'
y_share_bin = y_share.copy()
y_share_bin.loc[y_share_bin.isin(['decrease','constant'])] = 'decrease'

In [18]:
#randomly select 80% of nocs 100 times and aggregate the results
occ_workshop = data.index.drop_duplicates()
results = np.zeros([100,12])

for i in range(0,100):
    #select occupations
    train_occ_worshops = np.random.choice(occ_workshop,size=int(len(occ_workshop)*0.8),replace=False)
    test_occ_worshops = occ_workshop[np.logical_not(occ_workshop.isin(train_occ_worshops))]

    #select training and testing sets
    x_train = x.loc[train_occ_worshops]
    x_test = x.loc[test_occ_worshops]
    x_round_train = x_round.loc[train_occ_worshops]
    x_round_test = x_round.loc[test_occ_worshops]
    x_onehot_train = x_onehot.loc[train_occ_worshops]
    x_onehot_test = x_onehot.loc[test_occ_worshops]

    y_abs_train = y_abs.loc[train_occ_worshops]
    y_abs_test = abs_answer.loc[test_occ_worshops,['more','same','fewer']]
    y_share_train = y_share.loc[train_occ_worshops]
    y_share_test = share_answer.loc[test_occ_worshops,['increase','decrease','constant']]
    y_abs_bin_train = y_abs_bin.loc[train_occ_worshops]
    y_abs_bin_test = abs_answer.loc[test_occ_worshops,['more','not_increase']]
    y_share_bin_train = y_share_bin.loc[train_occ_worshops]
    y_share_bin_test = share_answer.loc[test_occ_worshops,['increase','not_increase']]

    #models!!
    gb_abs = GaussianNB().fit(x_train,y_abs_train)
    gb_share = GaussianNB().fit(x_train,y_share_train)
    gb_abs_bin = GaussianNB().fit(x_train,y_abs_bin_train)
    gb_share_bin = GaussianNB().fit(x_train,y_share_bin_train)
    mnb_abs = MultinomialNB().fit(x_round_train,y_abs_train)
    mnb_share = MultinomialNB().fit(x_round_train,y_share_train)
    mnb_abs_bin = MultinomialNB().fit(x_round_train,y_abs_bin_train)
    mnb_share_bin = MultinomialNB().fit(x_round_train,y_share_bin_train)
    mnb_abs_onehot = MultinomialNB().fit(x_onehot_train,y_abs_train)
    mnb_share_onehot = MultinomialNB().fit(x_onehot_train,y_share_train)
    mnb_abs_bin_onehot = MultinomialNB().fit(x_onehot_train,y_abs_bin_train)
    mnb_share_bin_onehot = MultinomialNB().fit(x_onehot_train,y_share_bin_train)
    
    #creating prediticting probabllitiy distributions over y variables
    gb_pred_abs = pd.DataFrame(mnb_abs.predict_proba(x_test)).set_index(x_test.index).loc[~x_test.index.duplicated(keep='first')]
    gb_pred_share = pd.DataFrame(mnb_share.predict_proba(x_test)).set_index(x_test.index).loc[~x_test.index.duplicated(keep='first')]
    gb_pred_abs_bin = pd.DataFrame(mnb_abs_bin.predict_proba(x_test)).set_index(x_test.index).loc[~x_test.index.duplicated(keep='first')]
    gb_pred_share_bin = pd.DataFrame(mnb_share_bin.predict_proba(x_test)).set_index(x_test.index).loc[~x_test.index.duplicated(keep='first')]
    pred_abs = pd.DataFrame(mnb_abs.predict_proba(x_round_test)).set_index(x_round_test.index).loc[~x_round_test.index.duplicated(keep='first')]
    pred_share = pd.DataFrame(mnb_share.predict_proba(x_round_test)).set_index(x_round_test.index).loc[~x_round_test.index.duplicated(keep='first')]
    pred_abs_bin = pd.DataFrame(mnb_abs_bin.predict_proba(x_round_test)).set_index(x_round_test.index).loc[~x_round_test.index.duplicated(keep='first')]
    pred_share_bin = pd.DataFrame(mnb_share_bin.predict_proba(x_round_test)).set_index(x_round_test.index).loc[~x_round_test.index.duplicated(keep='first')]
    pred_abs_onehot = pd.DataFrame(mnb_abs_onehot.predict_proba(x_onehot_test)).set_index(x_onehot_test.index).loc[~x_onehot_test.index.duplicated(keep='first')]
    pred_share_onehot = pd.DataFrame(mnb_share_onehot.predict_proba(x_onehot_test)).set_index(x_onehot_test.index).loc[~x_onehot_test.index.duplicated(keep='first')]
    pred_abs_bin_onehot = pd.DataFrame(mnb_abs_bin_onehot.predict_proba(x_onehot_test)).set_index(x_onehot_test.index).loc[~x_onehot_test.index.duplicated(keep='first')]
    pred_share_bin_onehot = pd.DataFrame(mnb_share_bin_onehot.predict_proba(x_onehot_test)).set_index(x_onehot_test.index).loc[~x_onehot_test.index.duplicated(keep='first')]

    #assign column names
    pred_abs.columns = pred_abs_onehot.columns = gb_pred_abs.columns = ['fewer','more','same']
    pred_share.columns = pred_share_onehot.columns = gb_pred_share.columns = ['constant','decrease','increase']
    pred_abs_bin.columns = pred_abs_bin_onehot.columns = pred_share_bin.columns = pred_share_bin_onehot.columns = gb_pred_abs_bin.columns = gb_pred_share_bin.columns = ['increase','not_increase']
     

    #testing choice accuracy
    results[i,0] = sum(y_abs_test.idxmax(axis=1)==gb_pred_abs.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,1] = sum(y_share_test.idxmax(axis=1)==gb_pred_share.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,2] = sum(y_abs_bin_test.idxmax(axis=1)==gb_pred_abs_bin.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,3] = sum(y_share_bin_test.idxmax(axis=1)==gb_pred_share_bin.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,4] = sum(y_abs_test.idxmax(axis=1)==pred_abs.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,5] = sum(y_share_test.idxmax(axis=1)==pred_share.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,6] = sum(y_abs_bin_test.idxmax(axis=1)==pred_abs_bin.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,7] = sum(y_share_bin_test.idxmax(axis=1)==pred_share_bin.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,8] = sum(y_abs_test.idxmax(axis=1)==pred_abs_onehot.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,9] = sum(y_share_test.idxmax(axis=1)==pred_share_onehot.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,10] = sum(y_abs_bin_test.idxmax(axis=1)==pred_abs_bin_onehot.idxmax(axis=1))/y_abs_bin_test.shape[0]
    results[i,11] = sum(y_share_bin_test.idxmax(axis=1)==pred_share_bin_onehot.idxmax(axis=1))/y_abs_bin_test.shape[0]
    
    #most important features for the onehot categories bin outcome
    

In [124]:
#Looping through the different model setups and running them on random testing/ training sets 100 times.
#outputs are the mean accuracies for each model
#randomly select 80% of noc-workshop cobination as a testing set:

occ_workshop = data.index.drop_duplicates()
results = np.zeros([3,4]) #structure to put the main in

train_occ_worshops = np.random.choice(occ_workshop,size=int(len(occ_workshop)*0.8),replace=False)
test_occ_worshops = occ_workshop[np.logical_not(occ_workshop.isin(train_occ_worshops))]

#setting different x and y setups to iterate through
x_train = [x.loc[train_occ_worshops], x_round.loc[train_occ_worshops], x_onehot.loc[train_occ_worshops]]
x_test = [x.loc[test_occ_worshops], x_round.loc[test_occ_worshops], x_onehot.loc[test_occ_worshops]]

y_train = [y_abs.loc[train_occ_worshops], y_share.loc[train_occ_worshops],
           y_abs_bin.loc[train_occ_worshops], y_share_bin.loc[train_occ_worshops]]
y_test = [abs_answer.loc[test_occ_worshops,['fewer','more','same']], 
           share_answer.loc[test_occ_worshops,['constant','decrease','increase']],
           abs_answer.loc[test_occ_worshops,['more','not_increase']], 
           share_answer.loc[test_occ_worshops,['increase','not_increase']]]

models = [naive_bayes.GaussianNB(),naive_bayes.MultinomialNB(),naive_bayes.BernoulliNB()]# three models for 3 feature sets

j=0
k=3

model = models[j].fit(x_train[j],y_train[k])
pred = pd.DataFrame(model.predict_proba(x_test[j])).set_index(x_test[j].index).loc[~x_test[j].index.duplicated(keep='first')]
#pred.columns = y_test[k].columns
print(x_test[j].index.unique().shape,'\n\n',y_test[k].index.shape,'\n\n',test_occ_worshops.shape,'\n\n')
#sum(pred.idxmax(axis=1) == y_test[k].idxmax(axis=1))/y_test[k].shape[0]

#for j in range(3):
#    for k in range(4):
#        model = models[j].fit(x_train[j],y_train[k])
#        pred = pd.DataFrame(model.predict_proba(x_test[j])).set_index(x_test[j].index)#.loc[~x_test[j].index.duplicated(keep='first')]
#        pred.columns = y_test[k].columns
#        print(pred.iloc[0,:],'\n\n',y_test[k].iloc[0,:],'\n\n')
        #results[j,k] = sum(pred.idxmax(axis=1) == y_test[k].idxmax(axis=1))/y_test[k].shape[0]

(24,) 

 (24,) 

 (24,) 




In [96]:
x.loc[test_occ_worshops]

value.Reading Comprehension  \
noc                                           workshop.number                                
Airline ticket and service agents             3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
                                              3                                   3.500000   
Court clerks                                  4                                   3.583333   
                                              4                                   3.583333   
                                              4                                   3.583333   
                                              4                                   3.583333   
                                              4                                   3.583333   
                                              4                                   3.583333   
                                              4                                   3.583333   
                                              4                                   3.583333   
                                              4                                   3.583333   
                                              4                                   3.583333   
...                                                                                    ...   
Underground production and development miners 3                                   2.781250   
                                              3                                   2.781250   
                                              3                                   2.781250   
                                              3                                   2.781250   
                                              3                                   2.781250   
                                              3                                   2.781250   
                                              3                                   2.781250   
                                              6                                   2.781250   
                                              6                                   2.781250   
                                              6                                   2.781250   
                     

In [74]:
abs_answer.loc[test_occ_worshops,['fewer','more','same']]

absolute                                                               fewer  \
noc                                                workshop.number             
Airline ticket and service agents                  3                0.750000   
Carpenters                                         3                0.100000   
Central control and process operators, petroleu... 1                0.619048   
Computer network technicians                       3                0.250000   
                                                   6                0.217391   
Court clerks                                       4                0.368421   
Forestry technologists and technicians             5                0.533333   
Furniture and fixture assemblers and inspectors    1                1.000000   
                                                   4                0.947368   
                                                   5                0.866667   
Industrial and manufacturing engineers             6                0.173913   
Instructors of persons with disabilities           2                0.043478   
                                                   4                0.210526   
Labourers in food, beverage and associated prod... 2                0.521739   
                                                   3                0.500000   
Medical administrative assistants                  2                0.608696   
                                                   3                0.400000   
Operators and attendants in amusement, recreati... 5                0.066667   
Plastics processing machine operators              1                0.904762   
                                                   2                0.608696   
                                                   4                0.947368   
Senior managers - financial, communications and... 3                0.100000   
                                                   4                0.157895   
Supervisors, petroleum, gas and chemical proces... 3                0.050000   

absolute                                                                more  \
noc                                                workshop.number             
Airline ticket and service agents                  3                0.100000   
Carpenters                                         3                0.650000   
Central control and process operators, petroleu... 1                0.000000   
Computer network technicians                       3                0.650000   
                                                   6                0.652174   
Court clerks                                       4                0.263158   
Forestry technologists and technicians             5                0.200000   
Furniture and fixture assemblers and inspectors    1                0.000000   
                                                   4                0.052632   
                                                   5                0.066667   
Industrial and manufacturing engineers             6                0.608696   
Instructors of persons with disabilities           2                0.652174   
                                                   4                0.473684   
Labourers in food, beverage and associated prod... 2                0.217391   
                                                   3                0.200000   
Medical administrative assistants                  2                0.173913   
                                                   3                0.400000   
Operators and attendants in amusement, recreati... 5                0.733333   
Plastics processing machine operators              1                0.000000   
                                                   2                0.130435   
                                                   4                0.000000   
Senior managers - financial, communications and... 3                0.500000   

In [11]:
agg_results = np.zeros([2,12])
for i in range(0,12):
    agg_results[0,i] = np.mean(results[:,i])
    agg_results[1,i] = np.std(results[:,i])

In [12]:
pd.DataFrame(agg_results,index=['mean','std'],columns=['abs','share','abs_bin','share_bin','abs_round','share_round','abs_round_bin','share_round_bin','abs_onehot','share_onehot','abs_bin_onehot','share_bin_onehot'])

,abs,share,abs_bin,share_bin,abs_round,share_round,abs_round_bin,share_round_bin,abs_onehot,share_onehot,abs_bin_onehot,share_bin_onehot
mean,0.598750,0.470417,0.259167,0.378750,0.605833,0.492083,0.264167,0.355833,0.615000,0.542083,0.298333,0.41375
std,0.089298,0.087658,0.089283,0.102756,0.086887,0.084717,0.089602,0.110620,0.079083,0.095469,0.092744,0.09794


In [285]:
occ_workshop = data.index.drop_duplicates()

train_occ_worshops = np.random.choice(occ_workshop,size=int(len(occ_workshop)*0.8),replace=False)
test_occ_worshops = occ_workshop[np.logical_not(occ_workshop.isin(train_occ_worshops))]

x_onehot_train = x_onehot.loc[train_occ_worshops]
x_onehot_test = x_onehot.loc[test_occ_worshops]

y_abs_bin_train = y_abs_bin.loc[train_occ_worshops]
y_abs_bin_test = abs_answer.loc[test_occ_worshops,['fewer','not_decrease']]

mnb_abs_bin_onehot = MultinomialNB().fit(x_onehot_train,y_abs_bin_train)

log_prob_abs_bin = pd.DataFrame(mnb_abs_bin_onehot.feature_log_prob_,index=['fewer','not_decrease'],columns=x_onehot.columns)

In [286]:
log_prob_abs_bin.sort_values(by='not_decrease',axis=1,ascending = False)

,Dynamic Flexibility_1.0,Selective Attention_3.0,Category Flexibility_3.0,Management of Material Resources_2.0,Flexibility of Closure_3.0,Far Vision_3.0,Perceptual Speed_3.0,Food Production_1.0,Fine Arts_1.0,Installation_1.0,...,Number Facility_4.0,Mathematical Reasoning_4.0,Reading Comprehension_2.0,Extent Flexibility_4.0,Static Strength_4.0,Equipment Maintenance_4.0,English Language_5.0,Mechanical_5.0,Personnel and Human Resources_5.0,Administration and Management_5.0
fewer,-4.829660,-4.806129,-4.814470,-4.901938,-4.93351,-4.971976,-4.964163,-5.016071,-4.894652,-5.117342,...,-8.721480,-8.721480,-8.070893,-8.070893,-8.070893,-11.206387,-11.206387,-11.206387,-11.206387,-11.206387
not_decrease,-4.811978,-4.821438,-4.856258,-4.976382,-4.98530,-4.988665,-4.989790,-4.990915,-5.002240,-5.010245,...,-9.582662,-9.582662,-11.086740,-11.086740,-11.086740,-11.779887,-11.779887,-11.779887,-11.779887,-11.779887
